In [1]:
import numpy as np
import pandas as pd
rng = np.random.default_rng(42)

In [2]:
raw_data = pd.read_csv('./Absenteeism-data.csv')
raw_data.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [3]:
df = raw_data.copy()
df.drop('ID', axis=1, inplace=True)
df.describe()

,Reason for Absence,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,19.411429,222.347143,29.892857,36.417143,271.801774,26.737143,1.282857,1.021429,0.687143,6.761429
std,8.356292,66.312960,14.804446,6.379083,40.021804,4.254701,0.668090,1.112215,1.166095,12.670082
min,0.000000,118.000000,5.000000,27.000000,205.917000,19.000000,1.000000,0.000000,0.000000,0.000000
25%,13.000000,179.000000,16.000000,31.000000,241.476000,24.000000,1.000000,0.000000,0.000000,2.000000
50%,23.000000,225.000000,26.000000,37.000000,264.249000,25.000000,1.000000,1.000000,0.000000,3.000000
75%,27.000000,260.000000,50.000000,40.000000,294.217000,31.000000,1.000000,2.000000,1.000000,8.000000
max,28.000000,388.000000,52.000000,58.000000,378.884000,38.000000,4.000000,4.000000,8.000000,120.000000


#### Extract month value and day of the week

In [4]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, format="%d/%m/%Y")


In [5]:
df['Month Value'] = df['Date'].dt.month
df['Day of the Week'] = df['Date'].dt.day_of_week
df.drop('Date', axis=1, inplace=True)
df.head()

,Reason for Absence,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month Value,Day of the Week
0,26,289,36,33,239.554,30,1,2,1,4,7,1
1,0,118,13,50,239.554,31,1,1,0,0,7,1
2,23,179,51,38,239.554,31,1,0,0,2,7,2
3,7,279,5,39,239.554,24,1,2,0,4,7,3
4,23,289,36,33,239.554,30,1,2,1,2,7,3


#### Education column into binary mapping

In [6]:
df['Education'].value_counts()

Education
1    583
3     73
2     40
4      4
Name: count, dtype: int64

In [7]:
df['Education'] = df['Education'].map({1:0, 2:1, 3:1, 4:1})

In [8]:
df['Education'].unique()

array([0, 1])

#### Split the reasons for absence column into dummy variable

In [9]:
print("unique values are ", sorted(df['Reason for Absence'].unique()))
print(len(df['Reason for Absence'].unique()))
# reason 20 is missing!

unique values are  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28]
28


#### from dummies, we should always remove one column to avoid multicollinearity. But how to decide which:
* If a person has been absent due to reason 0, this means they 
* have been away from work for an unknown reason. 
* Hence, this column acts like the baseline, 
* and all the rest are represented in comparison to this.
So in a nutshell, remove the column that acts as a baseline.

In [10]:
reason_columns = pd.get_dummies(df['Reason for Absence'], dtype=int, drop_first=True)
reason_columns.head()

,1,2,3,4,5,6,7,8,9,10,...,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [11]:
# 0: missing value
# 1: single value
# 2,3...: more than one reason for absence
reason_columns['check'] = reason_columns.sum(axis=1)
print(reason_columns['check'].unique())
print(reason_columns['check'].sum())
reason_columns.drop('check', axis=1)

[1 0]
662


,1,2,3,4,5,6,7,8,9,10,...,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


#### group dummies together
* reason 1-14 related to disease
* reason 15-17 related to pregnancy
* reason 18-21 related to poisoning and abnormalities
* reason 22-28 related to Light reasons for absence such as dental appointment, consultation, etc..

In [12]:
# since we already checked that there is one reason for absence only, here we can use max instead of sum to get
# the groupped column value. this value means that a person was absent because of one of the 
# reasons included in the cols 1-14
reason_type_1 = reason_columns.loc[:, 1:14].max(axis=1)
reason_type_2 = reason_columns.loc[:, 15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:, 18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:, 22:].max(axis=1)

In [13]:
df.shape

(700, 12)

In [14]:
reasons_df = pd.concat([reason_type_1, reason_type_2, 
                        reason_type_3, reason_type_4], 
                        axis=1)
df.drop(['Reason for Absence'], axis=1, inplace=True)

In [15]:
df = pd.concat([df, reasons_df], axis=1)
df.columns.values

array(['Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 'Month Value',
       'Day of the Week', 0, 1, 2, 3], dtype=object)

#### Rename the columns

In [16]:
df.columns = ['Transportation Expense', 'Distance to Work', 'Age',
              'Daily Work Load Average', 'Body Mass Index', 'Education',
              'Children', 'Pets', 'Absenteeism Time in Hours', 'Month Value', 'Day of the Week', 
              'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4',]

#### Reorder the columns

In [18]:
df = df[['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']]

#### Check point

In [19]:
df_preprocessed = df.copy()
df_preprocessed.shape

(700, 15)

In [20]:
df_preprocessed.to_csv('Absenteeism_preprocessed.csv', index=False)